In [85]:
import pandas as pd
from flask import Flask, request, jsonify
from apriori_python import apriori
import warnings
from flask_ngrok import run_with_ngrok

In [86]:
warnings.filterwarnings('ignore')

In [87]:
app = Flask(__name__)

In [88]:
run_with_ngrok(app)

In [89]:

df = pd.read_csv('data1.csv', encoding='unicode_escape')
df.rename(columns={
    "InvoiceNo": "invoice_num",
    "StockCode": "stock_code",
    "Description": "description",
    "Quantity": "quantity",
    "InvoiceDate": "invoice_date",
    "UnitPrice": "unit_price",
    "CustomerID": "customer_id",
    "Country": "country"
}, inplace=True)

df['invoice_date'] = pd.to_datetime(df['invoice_date'], format='%m/%d/%Y %H:%M')
df['description'] = df['description'].str.lower()
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df = df[df['quantity'] > 0]

In [90]:
transactions = df.groupby('invoice_num')['description'].apply(set).tolist()

In [91]:
min_support = 0.05
min_confidence = 0.1

In [92]:
freq_itemsets, rules = apriori(transactions, minSup=min_support, minConf=min_confidence)


In [93]:
indices = {item.lower().strip(): idx for idx, item in enumerate(df['description'].unique())}


In [94]:
rule_map = {}
for rule in rules:
    for item in rule[0]:  # For each item in the antecedent
        if item not in rule_map:
            rule_map[item] = []
        rule_map[item].append(rule)

In [95]:
@app.route('/recommend', methods=['POST'])
def recommend():
    try:
        
        input_product = request.json.get('product')
        
        if not input_product:
            return jsonify({'error': 'Please provide a valid product description.'}), 400

        
        input_product_normalized = input_product.lower().strip()

       
        if input_product_normalized not in indices:
            return jsonify({'error': 'Product not found in the dataset.'}), 404

        
        idx = indices[input_product_normalized]

        
        recommendations = set()

       
        for rule in rules:
            if set(rule[0]).issubset([input_product_normalized]) and not set(rule[1]).intersection([input_product_normalized]):
                recommendations.update(rule[1])

        return jsonify({
            'input_product': input_product,
            'recommendations': list(recommendations)
        }), 200

    except Exception as e:
        return jsonify({
            'error': 'An error occurred while processing your request.',
            'details': str(e)
        }), 500

In [96]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


 * Running on http://d7cd-110-227-212-246.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/Dec/2024 18:15:19] "POST /recommend HTTP/1.1" 200 -
